^C
Note: you may need to restart the kernel to use updated packages.


Looking in indexes: https://download.pytorch.org/whl/cu118
     ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
     - -------------------------------------- 0.2/6.2 MB 5.9 MB/s eta 0:00:02
     -- ------------------------------------- 0.4/6.2 MB 5.5 MB/s eta 0:00:02
     ---- ----------------------------------- 0.7/6.2 MB 5.2 MB/s eta 0:00:02
     ------ --------------------------------- 1.0/6.2 MB 5.6 MB/s eta 0:00:01
     -------- ------------------------------- 1.3/6.2 MB 5.5 MB/s eta 0:00:01
     ---------- ----------------------------- 1.6/6.2 MB 6.0 MB/s eta 0:00:01
     ----------- ---------------------------- 1.8/6.2 MB 5.8 MB/s eta 0:00:01
     ------------- -------------------------- 2.1/6.2 MB 5.6 MB/s eta 0:00:01
     --------------- ------------------------ 2.4/6.2 MB 5.7 MB/s eta 0:00:01
     ----------------- ---------------------- 2.7/6.2 MB 5.6 MB/s eta 0:00:01
     ------------------ --------------------- 2.9/6.2 MB 5.7 MB/s eta 0:00:01
     -------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

c:\Users\USER\anaconda3\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [4]:
# Data Preprocessing
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [5]:
# Load Dataset
train_dataset = datasets.ImageFolder(root='dest_folder/train', transform=transform)
test_dataset = datasets.ImageFolder(root='dest_folder/test', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [10]:
# Define CNN Model
class FaceMaskCNN(nn.Module):
    def __init__(self):
        super(FaceMaskCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 64 * 64, 128)
        self.fc2 = nn.Linear(128, 2)  # 2 classes (Masked, Unmasked)
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [11]:

# Model Training
model = FaceMaskCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [12]:
for epoch in range(5):
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

Epoch 1, Loss: 0.011164500378072262
Epoch 2, Loss: 0.004086191300302744
Epoch 3, Loss: 0.0054197851568460464
Epoch 4, Loss: 0.0040179090574383736
Epoch 5, Loss: 0.013314950279891491


In [13]:
# Save Model
torch.save(model.state_dict(), "facemask_model.pth")